In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
# from textblob import TextBlob

def preprocessing(lines):
    words = lines.select(explode(split(lines.value, "t_end")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    return words

In [2]:
# # text classification
# def polarity_detection(text):
#     return TextBlob(text).sentiment.polarity
# def subjectivity_detection(text):
#     return TextBlob(text).sentiment.subjectivity
# def text_classification(words):
#     # polarity detection
#     polarity_detection_udf = udf(polarity_detection, StringType())
#     words = words.withColumn("polarity", polarity_detection_udf("word"))
#     # subjectivity detection
#     subjectivity_detection_udf = udf(subjectivity_detection, StringType())
#     words = words.withColumn("subjectivity", subjectivity_detection_udf("word"))
#     return words

In [3]:
# create Spark session
spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()
# read the tweet data from socket
lines = spark.readStream.format("socket").option("host", "127.0.0.1").option("port", 5555).load()
# Preprocess the data
words = preprocessing(lines)

In [7]:
# # text classification to define polarity and subjectivity
# words = text_classification(words)
# words = words.repartition(1)
query = words.writeStream.queryName("all_tweets_2")\
    .outputMode("append").format("parquet")\
    .option("path", "./parc")\
    .option("checkpointLocation", "./check")\
    .trigger(processingTime='60 seconds').start()
query.awaitTermination()

KeyboardInterrupt: 

In [5]:
df = spark.read.parquet("./parc")
df.show()

+--------------------+
|                word|
+--------------------+
|       easy life ...|
|🤫 As a surprise ...|
|all my friends ge...|
|Have a a crazy li...|
|  Reimburse ’s lu...|
|Check out this li...|
|Addicted to this ...|
|            💥BANG🚀|
|🤫 As a surprise ...|
| Great way to put...|
|  Kindness is not...|
|Grease. Bye Bye B...|
|            💥BANG🚀|
| I don’t think an...|
|  Happy Monday &a...|
|If you need a sug...|
| Take the followi...|
|“I love it out th...|
|  You're not a pr...|
|            💥BANG🚀|
+--------------------+
only showing top 20 rows



In [10]:
df.info()

AttributeError: 'DataFrame' object has no attribute 'info'